In [15]:
import pandas as pd
import numpy as np

###  GOAL: Obtain a rate of  mental health providers for child enrollees by region

#### Load Data

In [5]:
# Medicaid enrollment by age and region
df1 = pd.read_csv("Medicaid_Program_Enrollment_by_Month___Beginning_2009.csv")

In [3]:
# Child providers 
df2 = pd.read_csv("child-providers-1.csv")

#### Clean Data

In [6]:
# rename columns
df1.columns = df1.columns.str.replace(" ","_").str.lower()
df1.head()

,eligibility_year,eligibility_month,economic_region,aid_category,dual_eligible,managed_care_vs._fee_for_service,plan_name,plan_type,gender,age_group,race,number_of_recipients
0,2009,1,Capital District,FHPLUS,NO,MMC,CDPHP,HMO/PHSP,Female,00-20,ASIAN,5
1,2009,1,Capital District,FHPLUS,NO,MMC,CDPHP,HMO/PHSP,Female,00-20,BLACK,13
2,2009,1,Capital District,FHPLUS,NO,MMC,CDPHP,HMO/PHSP,Female,00-20,HISPANIC,11
3,2009,1,Capital District,FHPLUS,NO,MMC,CDPHP,HMO/PHSP,Female,00-20,OTHER,3
4,2009,1,Capital District,FHPLUS,NO,MMC,CDPHP,HMO/PHSP,Female,00-20,WHITE,52


In [8]:
#Let's look only at the most recent numbers from August 2021
df1 = df1[(df1.eligibility_year == 2021) & (df1.eligibility_month == 8)]

In [10]:
#Add a district column to child provider list using county

In [2]:
capital = ['GREENE', 'COLUMNIA', 'ALBANY', "RENSSELAER", "SCHENECTADY", "SARATOGA", "WARREN", "WASHINGTON"]
central = ["OSWEGO", "ONONDAGA", "CORTLAND", "MADISON", "CAYUGA"]
western = ["NIAGARA", "ERIE", "CHAUTAUQUA", "CATTARAUGUS", "ALLEGANY"]
finger = ["ORLEANS", "GENESSEE", "WYOMING", "LIVINGSTON", "MONROE", "WAYNE", "ONTARIO", "SENECA", "YATES"]
mohawk = ["ONEIDA", "HERKIMER", "HAMILTON", "FULTON", "MONTGOMERY", "SCHOHARIE"]
north = ["ST. LAWRENCE", "JEFFERSON", "LEWIS", "FRANKLIN", "CLINTON", "ESSEX"]
southern = ["STEUBEN", "SCHUYLER", "TOMPKINS", "CHEMUNG", "TIOGA", "BROOME", "DELAWARE", "OTSEGO", "CHENANGO"]
nyc = ["BRONX", "NEW YORK", "QUEENS", "KINGS", "RICHMOND"]
li = ["NASSAU", "SUFFOLK"]
mid = ["WESTCHESTER", "ROCKLAND", "ORANGE", "SULLIVAN", "ULSTER", "DUTCHESS", "PUTNAM"]

def districtify (county):
    if county in capital:
        return "Capital District"
    elif county in central:
        return "Central"
    elif county in finger:
        return "Finger Lakes"
    elif county in li:
        return "Long Island"
    elif county in mid:
        return "Mid-Hudson"
    elif county in nyc:
        return "New York City"
    elif county in mohawk:
        return "Mohawk Valley"
    elif county in southern:
        return "Southern Tier"
    elif county in north:
        return "North Country"
    elif county in western:
        return "Western"

In [11]:
df2['economic_region'] = df2.county.apply(districtify)

In [12]:
df2.head()

,medicaid_provider_id,npi,provider_or_facility_name,medicaid_type,profession_or_service,provider_specialty,service_address,city,state_x,zip_code,...,enrollment_begin_date,next_anticipated_revalidation_date,file_date,medically_fragile_children_directory_ind,provider_email,primary_taxonomy,selected_taxonomy,state_y,license_number,economic_region
0,193339,1609910058,BARNES CYNTHIA DAYNELL,FFS,PHYSICIAN,CHILD PSYCHIATRY,352 7TH AVE RM 1109,NEW YORK,NY,10001-5012,...,07/01/2004,04/01/2024,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,109110,New York City
1,232320,1578569364,CANINO IAN A MD,OPRA,PHYSICIAN,CHILD PSYCHIATRY,1051 RIVERSIDE DR # 1812,NEW YORK,NY,10032-1007,...,10/01/2013,09/15/2019,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,123507,New York City
2,233550,1134359953,DEPAULA ROBERTO,FFS,PHYSICIAN,CHILD PSYCHIATRY,1249 5TH AVE,NEW YORK,NY,10029-4413,...,12/13/2013,07/01/2023,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,124580,New York City
3,314934,1497838296,SHAH MANOJ RAMESHCHANDRA,OPRA,PHYSICIAN,CHILD PSYCHIATRY,1 BROOKDALE PLZ BROOKDALE HOSP MED CENTER,BROOKLYN,NY,11212-3139,...,05/17/2017,01/11/2024,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,135666,New York City
4,423294,1861530446,DATTA MOITRI NATH,FFS,PHYSICIAN,CHILD PSYCHIATRY,100 N BROADWAY,IRVINGTON,NY,10533-1254,...,12/03/1979,09/15/2026,02/28/2022,N,NaN,Yes,2084P0804X - Psychiatry & Neurology Child & A...,NY,138997,Mid-Hudson


#### How many mental health providers by region?
First let's look at the total, indescriminate of specialty

In [13]:
df2.economic_region.value_counts()

New York City       150
Long Island          64
Mid-Hudson           47
Finger Lakes         27
Western              23
Capital District     18
Central              14
Southern Tier         5
Mohawk Valley         2
Name: economic_region, dtype: int64

In [23]:
region_providers = df2.groupby("economic_region").selected_taxonomy.value_counts().to_frame() \
.rename({'selected_taxonomy':'providers'},axis=1).reset_index()

In [57]:
region_providers

,economic_region,selected_taxonomy,providers
0,Capital District,2084P0804X - Psychiatry & Neurology Child & A...,12
1,Capital District,103TC2200X - Psychologist Clinical Child & Ad...,3
2,Capital District,261QM0855X - Clinic/Center Adolescent and Chi...,1
3,Capital District,3245S0500X - Substance Abuse Rehabilitation F...,1
4,Capital District,364SP0810X - Clinical Nurse Specialist Psych/...,1
5,Central,2084P0804X - Psychiatry & Neurology Child & A...,10
6,Central,103TC2200X - Psychologist Clinical Child & Ad...,4
7,Finger Lakes,2084P0804X - Psychiatry & Neurology Child & A...,17
8,Finger Lakes,103TC2200X - Psychologist Clinical Child & Ad...,9
9,Finger Lakes,364SP0810X - Clinical Nurse Specialist Psych/...,1


In [27]:
df1.head()

,eligibility_year,eligibility_month,economic_region,aid_category,dual_eligible,managed_care_vs._fee_for_service,plan_name,plan_type,gender,age_group,race,number_of_recipients
2194273,2021,8,Capital District,SSI,NO,FFS,FEE-FOR-SERVICE,FEE-FOR-SERVICE,Female,00-20,ASIAN,11
2194274,2021,8,Capital District,SSI,NO,FFS,FEE-FOR-SERVICE,FEE-FOR-SERVICE,Female,00-20,BLACK,41
2194275,2021,8,Capital District,SSI,NO,FFS,FEE-FOR-SERVICE,FEE-FOR-SERVICE,Female,00-20,HISPANIC,40
2194276,2021,8,Capital District,SSI,NO,FFS,FEE-FOR-SERVICE,FEE-FOR-SERVICE,Female,00-20,NATIVE AMERICAN,1
2194277,2021,8,Capital District,SSI,NO,FFS,FEE-FOR-SERVICE,FEE-FOR-SERVICE,Female,00-20,OTHER,74


In [61]:
region_child_enrollees = df1.query('age_group =="00-20"').groupby(["economic_region"]).number_of_recipients.sum().to_frame().reset_index()
region_child_enrollees

,economic_region,number_of_recipients
0,Capital District,96067
1,Central,85123
2,Finger Lakes,124357
3,Long Island,232259
4,Mid-Hudson,259197
5,Mohawk Valley,58962
6,New York City,1294219
7,North Country,41475
8,Other,730
9,Southern Tier,65519


In [30]:
### Merge the providers by region and the enrollees by region

region_merge = region_providers.merge(region_child_enrollees, how="left", left_on="economic_region", right_on="economic_region")

In [32]:
region_merge.head()

,economic_region,selected_taxonomy,providers,number_of_recipients
0,Capital District,2084P0804X - Psychiatry & Neurology Child & A...,12,96067
1,Capital District,103TC2200X - Psychologist Clinical Child & Ad...,3,96067
2,Capital District,261QM0855X - Clinic/Center Adolescent and Chi...,1,96067
3,Capital District,3245S0500X - Substance Abuse Rehabilitation F...,1,96067
4,Capital District,364SP0810X - Clinical Nurse Specialist Psych/...,1,96067


In [44]:
### Get a rate
region_merge['rate'] = (region_merge.number_of_recipients / region_merge.providers).round()

## Child Mental Health Providers By Region in order of worst rate (1 provider for every XXX children)

In [48]:
region_merge.sort_values(["selected_taxonomy",'rate'], ascending=False)

,economic_region,selected_taxonomy,providers,number_of_recipients,rate
9,Finger Lakes,364SP0810X - Clinical Nurse Specialist Psych/...,1,124357,124357.0
4,Capital District,364SP0810X - Clinical Nurse Specialist Psych/...,1,96067,96067.0
25,Southern Tier,364SP0810X - Clinical Nurse Specialist Psych/...,1,65519,65519.0
3,Capital District,3245S0500X - Substance Abuse Rehabilitation F...,1,96067,96067.0
13,Long Island,282NC2000X - General Acute Care Hospital Chil...,1,232259,232259.0
22,New York City,261QM0855X - Clinic/Center Adolescent and Chi...,1,1294219,1294219.0
28,Western,261QM0855X - Clinic/Center Adolescent and Chi...,1,152687,152687.0
16,Mid-Hudson,261QM0855X - Clinic/Center Adolescent and Chi...,2,259197,129598.0
12,Long Island,261QM0855X - Clinic/Center Adolescent and Chi...,2,232259,116130.0
2,Capital District,261QM0855X - Clinic/Center Adolescent and Chi...,1,96067,96067.0


In [58]:
psyc = ["2084P0804X  - Psychiatry & Neurology Child & Adolescent Psychiatry","103TC2200X  - Psychologist Clinical Child & Adolescent"]

In [59]:
### I'm mostly interested in psychologist and psychiatrists right now
region_merge[region_merge.selected_taxonomy.isin(psyc)].sort_values(["selected_taxonomy",'rate'], ascending=False)

,economic_region,selected_taxonomy,providers,number_of_recipients,rate
18,Mohawk Valley,2084P0804X - Psychiatry & Neurology Child & A...,1,58962,58962.0
24,Southern Tier,2084P0804X - Psychiatry & Neurology Child & A...,2,65519,32760.0
19,New York City,2084P0804X - Psychiatry & Neurology Child & A...,101,1294219,12814.0
26,Western,2084P0804X - Psychiatry & Neurology Child & A...,14,152687,10906.0
5,Central,2084P0804X - Psychiatry & Neurology Child & A...,10,85123,8512.0
14,Mid-Hudson,2084P0804X - Psychiatry & Neurology Child & A...,31,259197,8361.0
0,Capital District,2084P0804X - Psychiatry & Neurology Child & A...,12,96067,8006.0
7,Finger Lakes,2084P0804X - Psychiatry & Neurology Child & A...,17,124357,7315.0
10,Long Island,2084P0804X - Psychiatry & Neurology Child & A...,40,232259,5806.0
17,Mohawk Valley,103TC2200X - Psychologist Clinical Child & Ad...,1,58962,58962.0
